# MILES-GUESS Regression Example Notebook

John Schreck, David John Gagne, Charlie Becker, Gabrielle Gantos, Dhamma Kimpara, Thomas Martin

#### Objective

This notebook is meant to demonstrate a functional example of how to use the miles-guess repository for training an evidential regression model.

Steps to get this notebook running:
1) Follow package installation steps in the miles-guess [ReadMe](https://github.com/ai2es/miles-guess/tree/main).
2) Run the cells in this notebook in order.

#### Introduction

Evidential deep learning is capable of training a single model to simultaneously learn the problem and estimate the problem's aleatoric and epistemic uncertainties. This notebook represents an example of evidential deep learning for a regression problem.

In [1]:
import os, tqdm, yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler, RobustScaler

from mlguess.keras.models import GaussianRegressorDNN, EvidentialRegressorDNN
from mlguess.keras.models import BaseRegressor as RegressorDNN
from mlguess.keras.callbacks import get_callbacks
from mlguess.regression_uq import compute_results

2024-02-27 22:23:40.717721: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 22:23:40.757269: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 22:23:40.757304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 22:23:40.758607: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 22:23:40.766709: I tensorflow/core/platform/cpu_feature_guar

ModuleNotFoundError: No module named 'keras.ops'

### Load a configuation yaml 

In [ ]:
config = "../config/surface_layer/mlp.yml"

with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

### Load example training data from Zenodo

In [ ]:
data_seed = 0
flat_seed = 1000

n_splits = 1
n_models = 1

input_cols = conf["data"]["input_cols"]
output_cols = ['friction_velocity:surface:m_s-1']

In [ ]:
data = pd.read_parquet("https://zenodo.org/record/10719142/files/ptype_qc_10k.parquet")
data["day"] = data["Time"].apply(lambda x: str(x).split(" ")[0])

# Need the same test_data for all trained models (data and model ensembles)
flat_seed = 1000
gsp = GroupShuffleSplit(n_splits=1,  random_state=flat_seed, train_size=0.9)
splits = list(gsp.split(data, groups=data["day"]))
train_index, test_index = splits[0]
train_data, test_data = data.iloc[train_index].copy(), data.iloc[test_index].copy() 

# Make N train-valid splits using day as grouping variable
gsp = GroupShuffleSplit(n_splits=n_splits,  random_state=flat_seed, train_size=0.885)
splits = list(gsp.split(train_data, groups=train_data["day"]))
train_index, valid_index = splits[data_seed]
train_data, valid_data = train_data.iloc[train_index].copy(), train_data.iloc[valid_index].copy()  

In [ ]:
x_scaler, y_scaler = RobustScaler(), MinMaxScaler((0, 1))
x_train = x_scaler.fit_transform(train_data[input_cols])
x_valid = x_scaler.transform(valid_data[input_cols])
x_test = x_scaler.transform(test_data[input_cols])

y_train = y_scaler.fit_transform(train_data[output_cols])
y_valid = y_scaler.transform(valid_data[output_cols])
y_test = y_scaler.transform(test_data[output_cols])

### 1. Deterministic multi-layer perceptron (MLP) to predict some quantity

In [ ]:
conf["model"]["epochs"] = 1
conf["model"]["verbose"] = 1

In [ ]:
model = RegressorDNN(**conf["model"])
model.build_neural_network(x_train.shape[-1], y_train.shape[-1])

In [ ]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_valid, y_valid),
    callbacks=get_callbacks(conf, path_extend="")
)

##### Predict with the model

In [ ]:
y_pred = model.predict(x_test, y_scaler)

In [ ]:
mae = np.mean(np.abs(y_pred[:, 0]-test_data[output_cols[0]]))
mae

##### Create a Monte Carlo ensemble

In [ ]:
monte_carlo_steps = 10

In [ ]:
results = model.predict_monte_carlo(x_test, monte_carlo_steps, y_scaler)

In [ ]:
mu_ensemble = np.mean(results, axis=0)
var_ensemble = np.var(results, axis=0)

### 2. Predict mu and sigma with a "Gaussian MLP"

In [ ]:
config = "../config/surface_layer/gaussian.yml"
with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

conf["model"]["epochs"] = 1
conf["model"]["verbose"] = 1

In [ ]:
gauss_model = GaussianRegressorDNN(**conf["model"])
gauss_model.build_neural_network(x_train.shape[-1], y_train.shape[-1])

In [ ]:
gauss_model.fit(
    x_train,
    y_train,
    validation_data=(x_valid, y_valid),
    callbacks=get_callbacks(conf, path_extend=f"")
)

In [ ]:
mu, var = gauss_model.predict_uncertainty(x_test, y_scaler)

In [ ]:
# compute variance and std from learned parameters
#mu, var = gauss_model.calc_uncertainties(y_pred, y_scaler)

In [ ]:
mae = np.mean(np.abs(mu[:, 0]-test_data[output_cols[0]]))
print(mae, np.mean(var) ** (1/2))

In [ ]:
import seaborn as sns
sns.jointplot(x = test_data[output_cols[0]], y = mu[:, 0], kind = 'hex')
plt.xlabel('Target')
plt.ylabel('Predicted Target')
plt.show()

In [ ]:
sns.jointplot(x = mu[:, 0], y = np.sqrt(var)[:, 0], kind = 'hex')
plt.xlabel('Predicted mu')
plt.ylabel('Predicted sigma')
plt.show()

### 3. Compute mu, aleatoric, and epistemic quantities using the evidential model

In [ ]:
config = "../config/surface_layer/evidential.yml"

with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

conf["model"]["epochs"] = 5
conf["model"]["verbose"] = 1

In [ ]:
ev_model = EvidentialRegressorDNN(**conf["model"])
ev_model.build_neural_network(x_train.shape[-1], y_train.shape[-1])

In [ ]:
ev_model.fit(
    x_train,
    y_train,
    validation_data=(x_valid, y_valid),
    callbacks=get_callbacks(conf, path_extend=""))

In [ ]:
result = ev_model.predict_uncertainty(x_test, scaler=y_scaler)
mu, aleatoric, epistemic = result

In [ ]:
mae = np.mean(np.abs(mu[:, 0] - test_data[output_cols[0]]))
print(mae, np.mean(aleatoric)**(1/2), np.mean(epistemic)**(1/2))

In [ ]:
compute_results(test_data,
                output_cols,
                mu,
                np.sqrt(aleatoric),
                np.sqrt(epistemic))

### 4. Create a deep ensemble with the Gaussian model so that the law of total variance can be applied to compute aleatoric and epistemic

In [ ]:
config = "../config/surface_layer/gaussian.yml"
with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

conf["save_loc"] = "./"
conf["model"]["epochs"] = 1
conf["model"]["verbose"] = 0
n_splits = conf["ensemble"]["n_splits"]

In [ ]:
# make save directory for model weights
os.makedirs(os.path.join(conf["save_loc"], "cv_ensemble", "models"), exist_ok=True)

In [ ]:
data_seed = 0
gsp = GroupShuffleSplit(n_splits=1, random_state=flat_seed, train_size=0.9)
splits = list(gsp.split(data, groups=data["day"]))
train_index, test_index = splits[0]

In [ ]:
ensemble_mu = np.zeros((n_splits, test_data.shape[0], 1))
ensemble_var = np.zeros((n_splits, test_data.shape[0], 1))

for data_seed in tqdm.tqdm(range(n_splits)):
    data = pd.read_csv(fn)
    data["day"] = data["Time"].apply(lambda x: str(x).split(" ")[0])

    # Need the same test_data for all trained models (data and model ensembles)
    flat_seed = 1000
    gsp = GroupShuffleSplit(n_splits=1,  random_state=flat_seed, train_size=0.9)
    splits = list(gsp.split(data, groups=data["day"]))
    train_index, test_index = splits[0]
    train_data, test_data = data.iloc[train_index].copy(), data.iloc[test_index].copy()

    # Make N train-valid splits using day as grouping variable
    gsp = GroupShuffleSplit(n_splits=n_splits, random_state=flat_seed, train_size=0.885)
    splits = list(gsp.split(train_data, groups=train_data["day"]))
    train_index, valid_index = splits[data_seed]
    train_data, valid_data = train_data.iloc[train_index].copy(), train_data.iloc[valid_index].copy()

    x_scaler, y_scaler = RobustScaler(), MinMaxScaler((0, 1))
    x_train = x_scaler.fit_transform(train_data[input_cols])
    x_valid = x_scaler.transform(valid_data[input_cols])
    x_test = x_scaler.transform(test_data[input_cols])

    y_train = y_scaler.fit_transform(train_data[output_cols])
    y_valid = y_scaler.transform(valid_data[output_cols])
    y_test = y_scaler.transform(test_data[output_cols])

    model = GaussianRegressorDNN(**conf["model"])
    model.build_neural_network(x_train.shape[-1], y_train.shape[-1])

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        callbacks=get_callbacks(conf, path_extend="")
    )

    model.model_name = f"cv_ensemble/models/model_seed0_split{data_seed}.h5"
    model.save_model()

    # Save the best model 
    model.model_name = "cv_ensemble/models/best.h5"
    model.save_model()

    mu, var = model.predict_uncertainty(x_test, y_scaler)
    mae = np.mean(np.abs(mu[:, 0]-test_data[output_cols[0]]))

    ensemble_mu[data_seed] = mu
    ensemble_var[data_seed] = var

##### Use the method predict_ensemble to accomplish the same thing given pretrained models:

In [ ]:
model = GaussianRegressorDNN().load_model(conf)

In [ ]:
ensemble_mu, ensemble_var = model.predict_ensemble(x_test, scaler=y_scaler)

In [ ]:
epistemic = np.var(ensemble_mu, axis=0)
aleatoric = np.mean(ensemble_var, axis=0)

In [ ]:
print(epistemic.mean()**(1/2), aleatoric.mean()**(1/2))

In [ ]:
compute_results(test_data,
                output_cols,
                np.mean(ensemble_mu, axis=0),
                np.sqrt(aleatoric),
                np.sqrt(epistemic))

### 5. Use Monte Carlo dropout with the Gaussian model to compute aleatoric and epistemic uncertainties

In [ ]:
monte_carlo_steps = 10

ensemble_mu, ensemble_var = model.predict_monte_carlo(x_test,
                                                      monte_carlo_steps,
                                                      scaler=y_scaler)

In [ ]:
ensemble_epistemic = np.var(ensemble_mu, axis=0)
ensemble_aleatoric = np.mean(ensemble_var, axis=0)
ensemble_mean = np.mean(ensemble_mu, axis=0)